# Classifying trialwise CorrectGo and NoGo trials

There are a number of steps to this. Hopefully we can recycle previous code and be up fairly quickly!

1. Load beta data. Ideally this process should include a cache into a pure python object so we don't have to reload it each time.
2. Preprocess the data.
3. Do cross-validated training and testing. Ideally an inner loop to select best parameters, an outer loop to get cross-validated performance, and final training over all the data to get an image. The inner loop can be probably be handled within the package we use probably.

In [1]:
import sys
import os
import pandas as pd



sys.path.append(os.path.abspath("../../ml/"))
from apply_loocv_and_save import load_and_preprocess
from dev_utils import read_yaml_for_host
import warnings


config_data = read_yaml_for_host("sst_config.yml")



python initialized for apply_loocv_and_save
cpus available; cpus to use:
10 9
10


In [2]:
import multiprocessing
import math
import nibabel as nib
import nilearn as nl
from nilearn.decoding import DecoderRegressor,Decoder
from sklearn.model_selection import KFold,GroupKFold,LeaveOneOut
cpus_available = multiprocessing.cpu_count()

cpus_to_use = min(cpus_available-1,math.floor(0.9*cpus_available))
print(cpus_to_use)

9


In [3]:
from dev_wtp_io_utils import cv_train_test_sets, asizeof_fmt
from nilearn.decoding import DecoderRegressor,Decoder

In [4]:
nonbids_data_path = config_data['nonbids_data_path']
ml_data_folderpath = nonbids_data_path + "fMRI/ml"


## Set up the paradigm

In [5]:

def trialtype_resp_trans_func(X):
    return(X.trial_type)


## Loading beta data

beta data is generally written in `load_multisubject_brain_data_sst_w1.ipynb`.

We just have to load it.

In [6]:
brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_30subs_correct_cond_pfc.pkl'
warnings.warn("not sure if this file holds up--it was created in 2021; need to see if it's still valid")
train_test_markers_filepath = ml_data_folderpath + "/train_test_markers_20220818T144138.csv"

/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_25653/256071196.py:2: UserWarning: not sure if this file holds up--it was created in 2021; need to see if it's still valid
  warnings.warn("not sure if this file holds up--it was created in 2021; need to see if it's still valid")


In [7]:


all_subjects = load_and_preprocess(
    brain_data_filepath,
    train_test_markers_filepath,
    subjs_to_use = None,
    response_transform_func = trialtype_resp_trans_func,
    clean=None)

warnings.warn("the data hasn't been cleaned at any point. the fMRIPrep cleaning pipeline has been applied; nothing else has been.")


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


checked for intersection and no intersection between the brain data and the subjects was found.
there were 30 subjects overlapping between the subjects marked for train data and the training dump file itself.


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


test_train_set: 62918
brain_data_filepath: 173
pkl_file: 168
train_test_markers_filepath: 158
response_transform_func: 144
sys: 72
Brain_Data_allsubs: 48
clean: 16
subjs_to_use: 16
3171
3171
cleaning memory


/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_25653/1474138623.py:8: UserWarning: the data hasn't been cleaned at any point. the fMRIPrep cleaning pipeline has been applied; nothing else has been.
  warnings.warn("the data hasn't been cleaned at any point. the fMRIPrep cleaning pipeline has been applied; nothing else has been.")


In [28]:
#cut down to just 9 subjects
selected_subjs={}
print(all_subjects.keys())
num_subjs = 9
#select subjs
subjs = all_subjects['metadata']['subject'].unique()
subjs.sort()
selected_sub_ids=subjs[0:num_subjs]
#get index of selected subjs
selected_subjs_idx_bool = all_subjects['metadata']['subject'].isin(selected_sub_ids)
#get list of True indices
selected_subjs_idx = selected_subjs_idx_bool[selected_subjs_idx].index.tolist()


dict_keys(['X', 'y', 'groups', 'metadata', 'y_cat', 'y_int'])


In [30]:

subjs = all_subjects['metadata']['subject'].unique()
selected_subjs['X'] = nib.funcs.concat_images([all_subjects['X'].slicer[...,s] for s in selected_subjs_idx])
selected_subjs['y'] = all_subjects['y'][selected_subjs_idx_bool]


TypeError: cannot do slice indexing on RangeIndex with these indexers [0        True
1        True
2        True
3        True
4        True
        ...  
3166    False
3167    False
3168    False
3169    False
3170    False
Name: subject, Length: 3171, dtype: bool] of type Series

In [32]:
selected_subjs['y'] = all_subjects['y'][selected_subjs_idx_bool]
selected_subjs['groups'] = all_subjects['groups'][selected_subjs_idx_bool]
selected_subjs['metadata'] = all_subjects['metadata'][selected_subjs_idx_bool]

In [37]:
# get the PFC mask
mask_nifti = nib.load(ml_data_folderpath + '/prefrontal_cortex.nii.gz')

In [38]:
#convert the y array to an integer array representing the string values of the y array
selected_subjs['y_cat'] = selected_subjs['y'].astype('category')
selected_subjs['y_int']=selected_subjs['y_cat'].cat.codes

# Training

I'm going to start with `cv_train_test_sets` and see how that goes. It sems likely it'll have to be re-written somewhat, but it might be a good starting point.

I think I need to run this without all the extra scaffolding--just tresting the Decoder on the data until I get something sensible. At the very least we need to know the Decoder object is handling balanced classes correctly.

In [ ]:
# use add PFC mask.ipynb to figure out how to get a PFC mask onto this data.

In [39]:
dec_main = Decoder(standardize=True,cv=GroupKFold(3),scoring='roc_auc',n_jobs=cpus_to_use,mask=mask_nifti)
cv_results = cv_train_test_sets(
    trainset_X = selected_subjs['X'],
    trainset_y = selected_subjs['y_int'],
    trainset_groups = selected_subjs['metadata']['subject'],
    decoders = [dec_main],
    cv=KFold(n_splits=3) # we use KFold, not GroupKfold, because it's splitting on Group anyway
    )

Groups are the same.
fold 1 of 3
In order to test on a training group of 6 items, holding out the following subjects:['DEV015' 'DEV006' 'DEV009']. prepping fold data.... fitting.... 4.9 GiB. trying decoder 1 of 1. predicting. test score was:. 0.5122285267071023
fold 2 of 3
In order to test on a training group of 6 items, holding out the following subjects:['DEV010' 'DEV005' 'DEV011']. prepping fold data.... fitting.... 5.0 GiB. trying decoder 1 of 1. 

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/decoding/decoder.py:141: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 0.43445692883895126
fold 3 of 3
In order to test on a training group of 6 items, holding out the following subjects:['DEV012' 'DEV013' 'DEV014']. prepping fold data.... fitting.... 4.9 GiB. trying decoder 1 of 1. 

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/decoding/decoder.py:141: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 0.5957894736842105


In [40]:
cv_results_dict = {
    'test_scores':cv_results[0],
    'results':cv_results[1],
    'results_by_trainset_item':cv_results[2],
}

In [41]:
cv_results_dict['results_by_trainset_item']

,y,group,y_pred
0,1,DEV005,0
1,0,DEV005,0
2,0,DEV005,0
3,0,DEV005,1
4,0,DEV005,1
...,...,...,...
912,0,DEV015,1
913,0,DEV015,0
914,1,DEV015,0
915,0,DEV015,1


The following is derived from 3-fold cross-validation, and should indicate train/test performance classifying trials across subjects.

In [50]:
from sklearn.metrics import precision_recall_fscore_support
#get precision and recall
print(precision_recall_fscore_support(
    cv_results_dict['results_by_trainset_item']['y'],
cv_results_dict['results_by_trainset_item']['y_pred'].astype(int),average='macro'))

#get roc_auc
from sklearn.metrics import roc_auc_score
print(roc_auc_score(
    cv_results_dict['results_by_trainset_item']['y'],
    cv_results_dict['results_by_trainset_item']['y_pred']))

(0.5015090495368123, 0.5040220259845717, 0.4038898145712142, None)
0.5040220259845718
